In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Сігмоїд-функція (активаційна функція)

In [2]:
def sigmoid(x):
# -------------------Завдання-------------------
    return 1 / (1 + np.exp(-x))
# ----------------------------------------------

Похідна від сігмоїд-функції

In [3]:
def sigmoid_deriv(x):
# -------------------Завдання-------------------
    return sigmoid(x)*(1-sigmoid(x))
# ----------------------------------------------

Функції, що зменшують швидкість навчання

In [4]:
def reduce_exp(x):
    return (np.exp(x) - 1) / 2

def reduce_cosh(x):
    return np.cosh(x) - 1

In [5]:
weights = [np.array([[1.0, 1.0, 1.0, 1.0], [0.2, 0.4, 0.7, 0.5], [0.3, 0.5, 0.6, 0.9]]),
           np.array([[0.2], [0.4], [0.6], [0.8]])]

Функція для тренування нейромережі

In [6]:
# ----------------------Завдання----------------------
# Встановіть потрібну швидкість навчання learning_rate
def fit(X, y, learning_rate=0.5, epochs=100000):
# ----------------------------------------------------

    # До вхідних даних додається колонка х0 з одиницями, що є множниками для біасу
    # Тепер розмірність вхідного вектора [1, 3]
    ones = np.atleast_2d(np.ones(X.shape[0]))
    X = np.concatenate((ones.T, X), axis=1)

    for k in range(epochs):

        # Вхідний вектор обирається випадковим чином з тренувальної вибірки
        i = np.random.randint(X.shape[0])
        a = [X[i]]

        for l in range(len(weights)):

            # Вхід до шару
            dot_value = np.dot(a[l], weights[l])

            # Результат роботи активаційної функції
            activation = sigmoid(dot_value)
            a.append(activation)

        # Помилка
        error = y[i] - a[-1]

        # Похідна dE/dw для вихідного шару
        deltas = [error * sigmoid_deriv(a[-1])]

        # Похідна dE/dw для прихованого шару
        for l in range(len(a) - 2, 0, -1):
            deltas.append(deltas[-1].dot(weights[l].T)
                          * sigmoid_deriv(a[l]))

        # Порядок розташування похідних змінюється на протилежний,
        # тобто (вихідний шар)->(прихований шар) змінюється на (прихований шар)->(вихідний шар)
        deltas.reverse()

        # З кожною епохою швидкість навчання зменшується
        
        # ------------------Увага------------------
        # Оберіть та розкоментуйте одну з функцій
        #learning_rate = reduce_exp(learning_rate)
        # чи
        learning_rate = reduce_cosh(learning_rate)
        # -----------------------------------------

        # Зворотнє поширення помилки
        for i in range(len(weights)):
            layer = np.atleast_2d(a[i])
            delta = np.atleast_2d(deltas[i])
            weights[i] += learning_rate * layer.T.dot(delta)

Функція для прогнозу вихідних даних на основі вхідних

In [7]:
def predict(x):
    a = np.concatenate((np.ones(1).T, np.array(x)), axis=0)
    for l in range(0, len(weights)):
        a = sigmoid(np.dot(a, weights[l]))
    return a

In [8]:
%%time
# %%time - вбудована магічна функція для виводу результатів роботи всього "cell"-у

# Дані зчитуються із файла
df = pd.read_csv('./dataset/data.csv', sep=',')

# Вихідні дані відокремлюються від загального набору даних
labels = df['y']
df = df.drop(['y'], axis=1)

# З вхідних та вихідних наборів даних формуються тренувальні та тестові вибірки відповідно
x_train, x_test, y_train, y_test = train_test_split(
    df, labels, test_size=0.2, random_state=0)
x_train, x_test, y_train, y_test = x_train.values, x_test.values, y_train.values, y_test.values

y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

fit(x_train, y_train)

y_pred = np.array([[0.]])
for e in x_test:

            # Прогноз для тестових вхідних даних
    y_pred = np.vstack((y_pred, int(predict(e))))

y_pred = y_pred[1::].reshape(1, len(y_test))[0]
y_test = y_test.reshape(1, len(y_test))[0]

# Точність роботи нейромережі
print ("Accuracy score: %f \n" %accuracy_score(y_test, y_pred))

Accuracy score: 0.750000 

CPU times: user 3.92 s, sys: 14.7 ms, total: 3.93 s
Wall time: 3.93 s
